# 초기세팅

## 모듈 임포트

In [15]:
from typing import List, Dict, Optional, TypedDict, Any
from fastapi import FastAPI , Body ,HTTPException, APIRouter, Query 
import torch
import ast
import json
import uuid
import numpy as np
import psycopg2
from pgvector.psycopg2 import register_vector
from sentence_transformers import SentenceTransformer
from google.genai import types
from fastapi.middleware.cors import CORSMiddleware
import os
from langgraph.graph import StateGraph, END

## 모델 로딩

### 재미나이 로딩

In [16]:
# API 키 로딩
keys = []
def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                keys.append(value)

# API 키 로드 및 환경변수 설정
load_api_keys('api_key.txt')

In [17]:
def keyChange():
    global client
    from google import genai
    tt = keys.pop(0)
    keys.append(tt)
    client = genai.Client(api_key=tt)

keyChange() #무료 키 다썻으면 다른 키로 로딩하기

### 백터DB & 임베딩 모델 로딩

In [18]:
model_vector = SentenceTransformer('jhgan/ko-sroberta-multitask')
# 2. 원격 pgvector 연결 설정
def get_db_connection():
    db_info= dict()
    with open("db_info.txt","r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=",1)
                db_info[key] = value

    conn = psycopg2.connect(
        host=db_info['host'],
        database=db_info['database'],
        user=db_info['user'],
        password=db_info['password'],
        port=db_info['port']
    )
    register_vector(conn)
    return conn


g:\conda_envs\bigproject-ai\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Na\.cache\huggingface\hub\models--jhgan--ko-sroberta-multitask. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1137.10it/s, Materializing param=pooler.dense.weight]       

## State

In [38]:
class State(TypedDict, total = False):
    txt: str  #front
    ep_num : int  #front 
    subtitle : str #front
    check : dict
    work_id : str    #front
    user_id : str    #front
    인물 : dict
    세계 : dict
    장소 : dict
    사건 : dict
    물건 : dict
    집단 : dict

# 기능

## 백터 DB

### 임베딩 백터 : 택스트 데이터를 임베딩 벡터로 바꿔서 반환(768차원)

In [20]:
def get_vector(text):
    return model_vector.encode([text])[0]

### 백터 DB 검색(유사도)  
(카테고리 or *(전부), 유저 질문, user_id, work_id, 유사도 몇까지 출력할지, 몇개 출력할지)

In [21]:

def userQuestion(category,user_query,user_id,work_id,sim,limit):  # 태그(*,인물,장소,사건,세계,등)
    conn = get_db_connection()
    cur = conn.cursor()
    query_vector = get_vector(user_query)
    search_sql = """
    SELECT * FROM (
        SELECT 
            id,
            setting, 
            ep_num,
            1 - (embedding <=> %s::vector) AS similarity 
        FROM lorebooks 
        WHERE embedding IS NOT NULL 
            AND %s = ANY(user_id)
            AND work_id = %s
            AND (%s = '*' OR category = %s)
    ) sub
    WHERE similarity >= %s  -- 여기서 원하는 유사도 문턱값을 설정 (예: 0.3 이상)
    ORDER BY similarity DESC  
    LIMIT %s;
    """
    cur.execute(search_sql, (query_vector,user_id,work_id,category,category,sim,limit))
    result = cur.fetchall()
    cur.close()
    conn.close()
    return result

### DB 검색(일반)  
(카테고리, 검색할 대상, user_id, work_id, 유사도 몇까지 출력할지, 몇개 출력할지)

In [22]:
def select(category,keyword,user_id,work_id): #태그,키워드,타이틀,작가 4개 다있어야함
    conn = get_db_connection()
    cur = conn.cursor()
    search_sql= """
    SELECT 
        id,
        settings,
        ep_num
    FROM lorebooks 
    WHERE %s = ANY(user_id)
        AND work_id = %s
        AND category = %s
        AND keyword = %s
    """
    cur.execute(search_sql, (user_id,work_id,category,keyword))
    result = cur.fetchall()
    # print(result)
    cur.close()
    conn.close()
    return result

## LLM 

### 메시지 입력

In [23]:
count_change= 0
def genai(system_prompt,msg):
    global count_change
    try:
        response = client.models.generate_content(
            model= "gemini-2.5-flash-lite", #'gemini-3-flash',"gemini-2.5-flash",
            config=types.GenerateContentConfig(
                system_instruction=system_prompt, # 여기에 시스템 프롬프트를 넣습니다.
                temperature=0.1, # 데이터 추출이므로 낮게 설정
            ),
            contents=msg
        )
        return response.text
    except Exception as e:
        count_change +=1
        if count_change >=30:
            return "모두다씀"
        keyChange()
        return genai(system_prompt,msg)


# def genai(system_prompt,msg): 
#     global count_change
#     try:
#         response = client.models.generate_content(
#             model="gemma-3-27b-it", #gemma-3-12b, gemma-3-1b ,gemma-3-2b, gemma-3-4b
#             contents=system_prompt+msg,
#         )
#         return response.text
#     except Exception as e:
#         count_change +=1
#         if count_change >=30:
#             return "모두다씀"
#         keyChange()
#         return genai(system_prompt,msg)

### 카테고리 추출

In [24]:
def Categories(state: State) -> State:
    system_prompt="""### Role
너는 소설 원문에서 설정 데이터를 누락 없이 추출하여 구조화하는 전문 데이터 분석가입니다.

### Task
당신은 '본문'의 글을 정보를 추출하기 쉽도록 키워드를 추출하고자 합니다. 
요약한 글에서 추출할 정보는 '인물, 세계, 장소, 사건, 물건, 집단'에 관한 내용입니다.
해당 내용 보전을 중심으로 키워드를 찾아, 추출해야 할 목록을 딕셔너리{} 텍스트 형태로 만들고자 합니다.
세계는 해당 이야기를 이해하기 위한 설정 등을 뜻하는 세계관을 뜻합니다

### Output Format 
{
'인물':['인물1','인물2',...],
'세계':['규칙1','규칙2',...],
'장소':['장소1','장소2'],
'사건':['사건1','사건2'],
'물건':['물건1',....],
'집단':['집단1','집단2'],
}

### Constraints
- 특정 주인공 중심의 요약을 하지 마세요. 등장하는 키워드를 분류하세요.
- 불릿 포인트 및 볼드체를  사용하지 마세요.
- **본문에 명확히 나오지 않은 정보는 빈 배열 `[]`로 처리하세요.
- **수식어 제거: 키워드의 수식어는 모두 제거하고 핵심 키워드만 추출하세요.
- **동일 개체 통합: 동일한 대상을 지칭하는 단어는 가장 명확한 대표 명칭 하나로 통합하세요.
- **개체명 표준화: 개채명의 이명(Different Name)이 본문에 섞여 나올 경우, 가장 자주 언급되거나 공식적인 명칭으로 단일화하세요.
"""
    msg = f"본문: {state.get('txt','')} \n 위 본문을 분석하여 '인물, 세계, 장소, 사건, 물건, 집단' 정보를 ```json 형식으로 추출해줘."
    
    result = genai(system_prompt,msg)
    print("목록 추출 완료")
    print("=="*8)
    print(result)
    print("=="*8)
    try:
        st = result.index('{')
        ed = result.index('}')
        result = ast.literal_eval(result[st:ed+1])
    except:
        print("딕셔너리 변환 에러")
        state = Categories(state)
    return {**state,
            'check' : result
           }

### 1.설정 추출 - 인물

In [25]:
def charter(state: State) -> State:
    lis = state.get('check')
    lis = lis['인물']
    txt = state.get('txt')
    chart = state['인물']
    for ix in lis:
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 인물 설정집(Character Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 등장인물 {ix}의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
새로운 정보가 발견될 때마다 기존 설정과 대조할 수 있도록 매우 구체적이고 객관적인 수치와 묘사 위주로 추출해야 합니다.
본문에서 해당 정보가 있다면 "Unknown"을 삭제하고 정보를 str형식으로 적으세요
# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 추측하지 말고 본문에 명시된 텍스트 그대로를 기반으로 추출하세요.
"""+"""   
# JSON_Output_Format
{"""+f""" 
  "{ix}" """+""": [
    { "별명": "본문에 명시된 별명 (없으면 Unknown)",
      "과거": "간략한 배경 스토리(없으면 Unknown)",
      "종족": "종족 (없으면 Unknown)",
      "연령": "나이 또는 연령대 (없으면 Unknown)",
      "직업/신분": "신분 (없으면 Unknown)",
      "소속집단/가문": "소속 (없으면 Unknown)",
      "외형": "외모 묘사 (없으면 Unknown)",
      "성격": ["성격 키워드1 (없으면 Unknown)"],
      "기술/능력": [
        {"기술명": "능력 이름 (없으면 Unknown)", "숙련도/강도": "수준 (없으면 Unknown)", "상세효과": "효과설명 (없으면 Unknown)"}
      ],
      "인물관계": [
        {"대상이름": "상대방 이름(없으면 Unknown)", "관계": "관계 정의( 없으면 Unknown"), "상세내용": "관계 설명(없으면 Unknown)"}
      ]
    }
  ]
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요."""
        
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식을 마크다운 없이 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)

            result = json.loads(result)
            print("변환 성공!")
            chart[ix] = result[ix]
        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
        
    return {**state,
           '인물' :chart
           }

### 2.설정추출 - 세계관

In [26]:
def rule(state: State) -> State:
    lis = state.get('check')
    lis = lis['세계']
    txt = state.get('txt')
    rule = state['세계']
    for ix in lis:
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 세계관 설정집을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 미리 추출한 세계관 키워드 {ix}을/를 본문 없이 작품 속 세계의 규칙을 이해할 수 있도록 300자 이내로 설명하는 문장을 만들어 JSON 포맷으로 출력하세요.

# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 요약은 가능하지만 본문의 내용을 바탕으로 추측 없이 문장을 생성하세요.
2. 연관된 이야기 없이 대상에 대한 내용만으로 짧게 요약하여 문장을 생성하세요.

"""+"""   
# JSON_Output_Format
{"""+f""" "{ix}" """+""": "문장"}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 명확히 나오지 않은 정보는 "Unknown" 또는 빈 배열 `[]`로 처리하세요."""
        
        msg= f"규칙을 바탕으로 순수한 Json 텍스트 형식을 마크다운 없이 출력하세요."           
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)

            result = json.loads(result)
            print("변환 성공!")
            rule[ix] = result[ix]
        except:
            print("딕셔너리 변환 에러")
            print("result")
            #state = charter(state)
    
    return {**state,
           '세계' :rule
           }

### 3.설정추출 - 장소

In [27]:
def place(state: State) -> State:
    lis = state.get('check')
    lis = lis['장소']
    txt = state.get('txt')
    place = state['장소']
    for ix in lis:
        system_prompt=  f"""# Role
귀하는 소설의 원문을 분석하여 장소 설정집(Place Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, {ix}의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
새로운 정보가 발견될 때마다 기존 설정과 대조할 수 있도록 매우 구체적이고 객관적인 수치와 묘사 위주로 추출해야 합니다.
본문에서 해당 정보가 있다면 "Unknown"을 삭제하고 정보를 str형식으로 적으세요

# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 추측하지 말고 본문에 명시된 텍스트 그대로를 기반으로 추출하세요.
"""+"""   
# JSON_Output_Format
{"""+f""" 
  "{ix}" """+""": 
    { "별칭": "지역별칭 (없으면 Unknown)"
      "분위기": "지역의 분위기(없으면 Unknown)",
      "역사": "지역의 역사 또는 생성된 시기 (없으면 Unknown)",
      "위치": "지역의 위치 (없으면 Unknown)",
      "집단": ["해당 공간에 속한 소속 집단/가문(없으면 Unknown)"],
      "하부지역": ["이 장소에 속한 다른 지역 (없으면 Unknown)"]
    }
  
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 정보가 전혀 없는 항목만 "Unknown"으로 남겨두세요."""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
                    

        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            place[ix] = result[ix]

        except:
            print("딕셔너리 변환 에러")
            print("result")
            #state = charter(state)
    
    return {**state,
           '장소' :place
           }

### 4.설정추출 - 사건

In [28]:
def event(state: State) -> State:
    lis = state.get('check')
    lis = lis['사건']
    txt = state.get('txt')
    event = state['사건']
    
    for ix in lis:
        system_prompt= f"""# Role
귀하는 소설의 원문을 분석하여 사건 설정집(Place Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 사건 {ix}의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
설명에는 해당 사건에 관해 간략한 원인과 결과를 한줄요약하여 써야한다

# 본문
{txt}
"""+"""   
# JSON_Output_Format
{"""+f'"{ix}"'+""": "해당 사건에 관한 짧은 설명"}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
"""
        msg=f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            event[ix] = result[ix]

        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '사건' :event
           }

### 5.설정추출 - 아이템

In [29]:
def item(state: State) -> State:
    lis = state.get('check')
    lis = lis['물건']
    txt = state.get('txt')
    item = state['물건']
    
    for ix in lis:
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 아이템 설정(item Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.
본문에서 해당 정보가 있다면 "Unknown"을 삭제하고 정보를 str형식으로 적으세요

# Task
제공된 소설 본문을 읽고, 물건 {ix}의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
summary에는 물건의 성능과 역할에 대해 간략하게 적어야 합니다.
character에는 해당 사건의 메인 인물을 적어야 합니다.

# 본문
{txt}
"""+"""   
# JSON_Output_Format
{"""+f'"{ix}"'+""": {"설명":"해당 물건 관한 짧은 설명 (없으면 Unknown)",
"관련인물" :["관련된인물 (없으면 Unknown)"] }}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 관련인물은 사건에 등장하는 모든 인물이 아닌 사건의 주역만 적어야 합니다. 주역이 없을 경우 빈 배열 '[]'로 처리하세요.
"""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            item[ix] = result[ix]

        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '물건' :item
           }

### 6.설정추출 - 단체

In [30]:
def group(state: State) -> State:
    lis = state.get('check')
    lis = lis['집단']
    txt = state.get('txt')
    group = state['집단']
    
    for ix in lis:
        system_prompt=f"""# Role
귀하는 소설의 원문을 분석하여 집단 설정(group Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.
본문에서 해당 정보가 있다면 "Unknown"을 삭제하고 정보를 str형식으로 적으세요

# Task
제공된 소설 본문을 읽고, 집단 {ix}의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
summary에는 집단의 역사와 목적 대해 간략하게 적어야 합니다.
character에는 해당 집단에 속해있는 인물을 적은 후 집단에서 역할을 괄호'()'안에 적으세요.

# 본문
{txt}
"""+"""   
# JSON_Output_Format
{"""+f'"{ix}"'+""": {"설명":"집단에 관한 짧은 설명(없으면 Unknown)",
"구성원" :["구성원(역할)(없으면 Unknown)"] }}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 정보가 전혀 없는 항목만 "Unknown"으로 남겨두세요.
- key값이나 character 의 구성원 value값은 오타나 언어의 변환 없이 본문 그대로 넣으세요.
"""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            group.update(result)

        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '집단' :group
           }

### 추출한 설정 검수 & 분류

In [31]:
def comparison(state: State):
    check = state.get('check')
    work_id = state.get('work_id')
    user_id = state.get('user_id')
    uplist ={'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    editlist = {'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    errorlist = {'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    oldlist = {'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    for tag,val in check.items():
        for keyword in val:
            emb = select(tag,keyword,title,userid)
            if (len(emb)==0):
                uplist[tag].append({keyword:state.get(tag).get(keyword),"ep_num":[state.get("ep_num")]}) 
                continue
            oldlist[tag].append(emb)
            i = state.get(tag).get(keyword)
            system_prompt= f"""#### 역할: 소설 설정 검수 전문가
        
### 상황
하나의 대상에 대하여 서로 다른 에피소드에서 뽑아온 설정이 있습니다. 해당 두 설정을 확인하고 설정이 어긋나는 경우가 존재하는지 찾아보세요.

### 분석할 설정
기존설정 : {emb[0][1]}

신규설정 : {i}

### 요청 사항
2. **상세 분석:** - **인과관계/상세화:** 
   - **결합:** 한쪽이 원인이고 다른 쪽이 결과이거나, 한쪽이 다른 쪽에서 없는 설명인 경우 '결합'로 판단합니다.
   - **충돌:** 예외 없이 물리적으로 불가능한 경우(예: 죽었는데 살아있음, 파란 눈인데 빨간 눈임)에만 '충돌'로 판단하세요.
    -**일치:** 설정이 추가되거나 보강되는 부분 없이 동일한 경우에는 '일치'로 판단하세요.
    
### 출력 형식
[결과: 충돌/결합/일치]
[판단사유: 판단사유]
        """
            msg = f"이거 설정이 맞을까?"     
            result = genai(system_prompt,msg)
            if "[결과: 충돌]" in result:
                errorlist[tag].append({keyword:result,"신규설정":i})
            elif "[결과: 결합]" in result:
                editlist[tag].append(emb)
            print(result)
    editlist = edit_setting(editlist,state)
    return {'충돌': errorlist, '설정 결합' : editlist,
            "신규 업로드": uplist, "기존설정" : oldlist}

In [32]:
def edit_setting(editlist, state:State):
    lis = []
    for tag,value in editlist.items():
        for emb in value:            
            vctid,oldset,episode = emb[0]
            keyword = next(iter(oldset))
            system_prompt= f"""#### 역할: 소설 설정 작성 전문가
            
    ### 상황
    하나의 대상에 대하여 서로 다른 에피소드에서 뽑아온 설정이 있습니다. 해당 두 설정을 확인하고 두 설정을 결합하세요.
    
    ### 분석할 설정
    기존설정 : {oldset}
    
    신규설정 : {keyword} : {state.get(tag).get(keyword)}
    
    ### 출력 형식
    설정 형식과 동일
    """
            msg = f"두 설정을 결합해줄래"     
            result = genai(system_prompt,msg)
            try:
                result = result.replace("json", "").replace("```", "").strip()
                result = json.loads(result)
            except:
                print("딕셔너리 변환 에러 : edit_setting")
                print(result)
                #state = charter(state)
            print(result)
            episode.append(state.get('episode'))
            lis.append([vctid,result,episode])
    return lis

## 원문&설정 읽기 쓰기

In [33]:
def save_text(user_id, work_id, ep_num, txt):
    try:
        save_dir = f"./save_original/{user_id}/{work_id}"
        os.makedirs(save_dir, exist_ok=True)
        file_path = os.path.join(save_dir, f"{ep_num}.txt")
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(txt)
            f.close()
        return f"./save_original/{user_id}/{work_id}/{ep_num}.txt"
    except Exception as e:
        print("error occured: ", e)
        return f"failed"

def open_text(user_id, work_id, ep_num) -> str:
    try:
        file_path = f"./save_original/{user_id}/{work_id}/{ep_num}.txt"
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e: 
        print("error occured: ", e)
        return "failed"
        
def save_json(user_id, work_id,lore_id, data):
    try:
        save_dir = f"./save_setting/{userid}/{work_id}"
        os.makedirs(save_dir, exist_ok=True)
        file_path = os.path.join(save_dir, f"{lore_id}.json")
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        return f"./save_setting/{userid}/{work_id}/{lore_id}.json"
    except Exception as e:
        print("error occured: ", e)
        return "failed"

def open_json(user_id, work_id,lore_id) :
    try:
        file_path = f"./save_original/{user_id}/{work_id}/{lore_id}.json"
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e: 
        print("error occured: ", e)
        return "failed"

## Rang Graph

### 카테고리

In [34]:
cat_builder = StateGraph(State)
cat_builder.add_node("Categories",Categories)

cat_builder.set_entry_point("Categories")
cat_builder.add_edge("Categories",END)

cat = cat_builder.compile()

### 상세설정

In [35]:
builder = StateGraph(State)
builder.add_node("charter",charter)
builder.add_node("rule",rule)
builder.add_node("place",place)
builder.add_node("event",event)
builder.add_node("item",item)
builder.add_node("group",group)

builder.set_entry_point("charter")
builder.add_edge("charter","rule")
builder.add_edge("rule","place")
builder.add_edge("place","event")
builder.add_edge("event","item")
builder.add_edge("item","group")
builder.add_edge("group",END)

detail = builder.compile()


# FastAPI

In [36]:
app = FastAPI()
users : Dict[str, State] = {}
origins = ["https://ai0917-kt-aivle-shool-8th-bigprojec-eta.vercel.app",
          "hppt://localhost:8080"]
app.add_middleware(
CORSMiddleware,
allow_origins=origins,
allow_credentials=True,
allow_methods=["GET", "POST", "PUT", "DELETE"],  
allow_headers=origins,
)
@app.get("/")
def connect_check():
    return "AI 서버와 연결되었습니다."

#원문 저장 
@app.post("/novel_save")
async def process_novel_save(data : dict = Body(...)):
    user_id = data.get("user_id")
    work_id = data.get("work_id")
    ep_num = data.get("ep_num")
    txt = data.get("txt")
    return save_text(user_id, work_id, ep_num, txt)

# 원문 읽기
@app.get("/novel_read")
async def process_novel_read(
    user_id: str = Query(...), 
    work_id: str = Query(...), 
    ep_num: int = Query(...)
):
    # 기존 코드의 txt 변수가 정의되어 있지 않아 에러가 날 수 있으니 확인이 필요해요!
    return open_text(user_id, work_id, ep_num)

#설정 저장
@app.post("/setting_save")
async def process_setting_save(data : dict = Body(...)):
    user_id = data.get("user_id")
    work_id = data.get("work_id")
    lore_id = data.get("lore_id")
    setting = data.get("data")
    return save_json(user_id,work_id,lore_id,setting)

# 설정 읽기
@app.get("/setting_read")
async def process_setting_read(
    user_id: str = Query(...), 
    work_id: str = Query(...), 
    lore_id: str = Query(...)
):
    return open_json(user_id, work_id, lore_id)

#카테고리 추출 
@app.post("/categories")
async def process_cat(data : dict = Body(...)):
    global current_state
    initial_state : State = {
        "txt": open_text(data.get("user_id"),data.get("work_id"),data.get("ep_num")),
        "ep_num" : data.get("ep_num"),
        "subtitle" : data.get("subtitle"),
        "check" : dict(),
        "work_id" : data.get("work_id"),
        "user_id" : data.get("user_id"),
        "인물" : dict(),
        "세계" : dict(),
        "장소" : dict(),
        "사건" : dict(),
        "물건" : dict(),
        "집단" : dict(),
    }
    if initial_state['txt'] == "failed":
        return "Error 원문이 저장되어 있지 않습니다."
    userid = initial_state.get('work_id')
    result = cat.invoke(initial_state)
    users[userid] = result
    return result.get("check",{})

@app.post("/setting") # 임베딩 벡터 반환하는 코드도 작성하기
async def process_get_setting(data : dict = Body(...)):
    userid = data.get("work_id")
    categories = data.get("check", [])
    if userid not in users:    # 존재하는 세션인지 확인
        raise HTTPException(status_code=404, detail="세션이 만료되었거나 존재하지 않습니다.")
    user_state = users[userid]  #기존의 데이/터 가져오기
    user_state["check"] = categories
    user_state = detail.invoke(user_state)
    result = comparison(user_state)
    return result 

app.post("/comparison") # 임베딩 벡터 반환하는 코드도 작성하기
async def process_get_comparison(data : dict = Body(...)):
    result = comparison(data)
    return result 

@app.post("/get_vector") # 임베딩 벡터 반환하는 코드도 작성하기
async def process_get_vector(data : dict = Body(...)):
    original = data.get("original")
    txt_original = f"{original}"
    return get_vector(txt_original)

#유사도 기반 vectorDB 검색
@app.post("/userQ")
async def process_userQ(data : dict = Body(...)):
    category = data.get("category")
    user_query = data.get("user_query")
    user_id = data.get("user_id")
    work_id = data.get("work_id")
    sim = data.get("sim")
    limit = data.get("limit")
    return userQuestion(category,user_query,user_id,work_id,sim,limit)



In [ ]:
import os
import sys
import subprocess
from datetime import datetime
from fastapi import HTTPException

@app.post("/webnovel_trend_report")
async def run_webnovel_trend_report():
    """
    트렌드 분석 보고서 생성 엔드포인트입니다. 
    요청 시 바로 반환을 보내며, subprocess를 생성해 webnovel_trend_analysis_and_report.py 스크립트를 실행합니다.
    로그는 logs폴더 내에 저장됩니다. 
    """

    script_path = os.path.join(os.getcwd(), "webnovel_trend_analysis_and_report.py")
    if not os.path.exists(script_path):
        raise HTTPException(status_code=404, detail=f"script not found: {script_path}")
    try:
        logs_dir = os.path.join(os.getcwd(), "logs")
        os.makedirs(logs_dir, exist_ok=True)
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_path = os.path.join(logs_dir, f"webnovel_trend_{ts}.log")
        env = os.environ.copy()
        env["PYTHONUNBUFFERED"] = "1"
        f = open(log_path, "wb")
        proc = subprocess.Popen(
            [sys.executable, "-u", script_path],
            cwd=os.getcwd(),
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            env=env,
        )
        # Stream bytes so progress bars without newlines still show up in the log file.
        def _pump():
            try:
                for chunk in iter(lambda: proc.stdout.read(1), b""):
                    f.write(chunk)
                    f.flush()
            finally:
                f.close()
        import threading
        threading.Thread(target=_pump, daemon=True).start()
        return {
            "started": True,
            "pid": proc.pid,
            "log_path": log_path,
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


NameError: name 'app' is not defined